# Finite state automata

by Koenraad De Smedt at UiB

---
A *finite state automaton* (FSA) is a formal computational device that recognizes strings belonging to a formal language. The descriptive capacity is equivalent to that of regular expressions. See also the page about FSA in the LING123 modules.

This notebook presents simple Python programs that accept strings according to a

1.   deterministic FSA
2.   non-deterministic FSA

Its purpose is to show step by step how an FSA works.

---

## 1. Deterministic FSA

Consider a simple deterministic finite state automaton that recognizes the language `a+b?`

<img src="https://git.app.uib.no/desmedt/teaching/-/raw/main/dfsa-ab.png" alt = "dfsa" width = 450px>

Symbols that are input to an FSA can be represented as characters in a string. Transitions can be represented in a dict. With every state in that dict, we associate another dict: every symbol that can be read from that state has the new state for that symbol.
Also states without any departing transitions must be in the dict.

In [ ]:
dfsa1 = {'q0': {'a':'q1',},
         'q1': {'a':'q1','b':'q2'},
         'q2': {}}

Here is a function that attempts to accept a string given a deterministic FSA. As arguments, the function takes a string of symbols, a dict with transitions, an initial state and a set of final states.

During computation, the function prints out every symbol that is read from the input string, as well as the new state, if any. If no transition is found, the function returns `False`. When all symbols are read, the result is `TRUE` if a final state is reached, `FALSE` otherwise.

In [ ]:
def accepts(s, trans, state, final):
  print('Initial state is', state)
  for c in s:
    print('Reading', c)
    state = trans[state].get(c)
    if state:
      print('- moving to state', state)
    else:
      print('- no new state found')
      return False
  return state in final

Call the function with a string of symbols, a transition dict, an initial state and a set of final states. Here are some tests.

In [ ]:
accepts('aa', dfsa1, 'q0', {'q1','q2'})

In [ ]:
accepts('aaab', dfsa1, 'q0', {'q1','q2'})

In [ ]:
accepts('aba', dfsa1, 'q0', {'q1','q2'})

In [ ]:
accepts('b', dfsa1, 'q0', {'q1','q2'})

The following is my recursive solution, which I personally consider more elegant. You can do the same tests as above.

In [ ]:
def accepts (s, trans, state, final):
  if not state:
    return False
  print('State is', state, '- string is', s)
  if not s:
    return state in final
  else:
    return accepts(s[1:], trans, trans[state].get(s[0]), final)

## 2. Non-deterministic FSA

Consider the following non-deterministic variant for the same language.

<img src="https://git.app.uib.no/desmedt/teaching/-/raw/main/nfsa-ab.png" alt = "nfsa-ab" width = 450px>

From a state, several transitions with the same symbol can depart. From *q0* in this automaton, reading symbol *a* gives two new nodes, *q0* and *q1*. In the dict, we will therefore require that with each symbol, a *list* of new states will be associated.

In [ ]:
nfsa1 = {'q0': {'a':['q1', 'q0']},
         'q1': {'b':['q2']},
         'q2': {}}

With such non-determinism, we cannot simply proceed from one state to the next. Instead, we need to try moving from several states onwards.

Therefore we need not only a recursion over the string of symbols, but also a recursion over a list of states. Make a new recursive function `accepts_l` that takes a list of states instead of a single state for its third parameter.

Then redefine `accepts` so that it puts the initial state in a list when it calls `accepts_l`.

In [ ]:
def accepts_l (s, trans, states, final):
  if not states:
    return False
  print('States are', states, '- string is', s)
  if not s:
    return set(states) & final # intersection non-empty?
  else:
    return (accepts_l (s[1:], trans, trans[states[0]].get(s[0]), final)
            or accepts_l (s, trans, states[1:], final))

def accepts (s, trans, state, final):
  return accepts_l(s, trans, [state], final)

The new `accepts` function can be used in the same way as the old one, except that we give it a FSA in the new format which is potentially non-deterministic. The result, if not False, will consist of the current nodes which are also final states.

In [ ]:
accepts('aa', nfsa1, 'q0', {'q1', 'q2'})

In [ ]:
accepts('aaab', nfsa1, 'q0', {'q1', 'q2'})

In [ ]:
accepts('aba', nfsa1, 'q0', {'q1', 'q2'})

In [ ]:
accepts('b', nfsa1, 'q0', {'q1', 'q2'})

### Exercises

1.  Write a deterministic FSA for the regexp `a+[bc]+` and test acceptance of several strings.
2.  Extend the non-deterministic FSA to `a+b+` by adding a loop with *b* at *q1*, which causes additional non-determinism. Perform tests.
3.  (optional) Write a program that converts a representation of transitions as shown in "1. Deterministic FSA" to the format shown in "2. Non-deterministic FSA".